In [ ]:
%run __init__.ipynb

In [ ]:
# Load data source
DB_TREND = get_datasource('WORLD_DB_TREND')
# db_trend

DB_ALL = get_datasource('WORLD_DB_ALL')
DB_ALL

In [ ]:
# # Get top 10 countries
# MAX_DATE = DB_TREND['DATE_ORDER'].max()
# FILTERS =  ['Europe', 'Asia', 'Africa', 'North America',
#             'South America', 'Oceania', 'Other', 'To be affected']
# COUNTRIES = (DB_TREND[(DB_TREND['ENTITY_GROUPS'].isin(FILTERS)) &
#                       (DB_TREND['DATE_ORDER'] == MAX_DATE) &
#                       (DB_TREND['KPI'] == 'Confirmed') &
#                       (DB_TREND['DATE_SCENARIO'] == 'Since beginning') &
#                       (DB_TREND['METRIC'] == 'VALUE')].
#              sort_values(by='VALUE', ascending=False)
#              ['ENTITY'].drop_duplicates().values.tolist()[:10])
# print(COUNTRIES)

In [ ]:
# Get top 10 countries
MAX_DATE = DB_ALL['DATE_ORDER'].max()
FILTERS = ['Europe', 'Asia', 'Africa', 'North America',
           'South America', 'Oceania', 'Other', 'To be affected']
COUNTRIES = (DB_ALL[(DB_ALL['ENTITY_GROUPS'].isin(FILTERS)) &
                    (DB_ALL['DATE_ORDER'] == MAX_DATE) &
                    (DB_ALL['KPI'] == 'Confirmed')].
             sort_values(by='VALUE', ascending=False)
             ['ENTITY'].drop_duplicates().values.tolist()[:10])
print(COUNTRIES)

In [ ]:
DB_LOGA = DB_ALL.copy()

In [ ]:
FILTERS = ['WORLDWIDE', 'Europe', 'Asia', 'Africa', 'North America',
           'South America', 'Oceania', 'Other', 'To be affected']
KPIS = ['Confirmed', 'Active cases', 'Deaths', 'Recovered']
DOMAIN_LOGA = DB_LOGA.copy()[(DB_LOGA['ENTITY_GROUPS'].isin(FILTERS)) &
                             (DB_LOGA['KPI'].isin(KPIS))]
DOMAIN_LOGA = DOMAIN_LOGA.drop(['DATE'],
                               axis=1).sort_values(by=['ENTITY', 'DATE_ORDER'],
                                                   ascending=True)
DOMAIN_LOGA

In [ ]:
# Select top 10 countries
DOMAIN_LOGA = DOMAIN_LOGA[(DOMAIN_LOGA['ENTITY'].isin(COUNTRIES))]

FILTERS = DOMAIN_LOGA[['ENTITY', 'KPI']].drop_duplicates().values.tolist()
DOMAIN104 = pd.DataFrame()
for fil in FILTERS:
    ENTITY = fil[0]
    KPI = fil[1]
    TMP_DF = DOMAIN_LOGA[(DOMAIN_LOGA['ENTITY'] == ENTITY) &
                         (DOMAIN_LOGA['KPI'] == KPI)]
    TMP_DF = TMP_DF.sort_values(by='DATE_ORDER',
                                ascending=True).reset_index(drop=True)

    # Calc if Confirmed
    if KPI == 'Confirmed':
        CHECK_LIMIT = TMP_DF.loc[TMP_DF['VARV'] > 30, 'VARV'].index
    elif KPI == 'Deaths':
        CHECK_LIMIT = TMP_DF.loc[TMP_DF['VARV'] > 3, 'VARV'].index
    elif KPI == 'Active cases':
        CHECK_LIMIT = TMP_DF.loc[TMP_DF['VARV'] > 50, 'VARV'].index
    elif KPI == 'Recovered':
        CHECK_LIMIT = TMP_DF.loc[TMP_DF['VARV'] > 30, 'VARV'].index

    if not CHECK_LIMIT:
        next
    else:
        LIMIT = CHECK_LIMIT[0]
        INDEXES = TMP_DF.loc[TMP_DF.index < LIMIT, 'VARV'].index
        TMP_DF = TMP_DF.drop(INDEXES, axis=0).reset_index(drop=True)

    TMP_DF['INDEX'] = TMP_DF.index
    TMP_DF['DATE'] = 'D' + TMP_DF['INDEX'].astype(str)

    def calc_ave(row, dataf):
        """
        doc string
        """
        i_min = row['INDEX'] - 6
        i_max = row['INDEX'] + 1
        list_filters = []
        for idx in range(i_min, i_max):
            list_filters.append(str(idx))
        dataf = dataf[dataf['INDEX'].isin(list_filters)]
        average = dataf['VARV'].sum() / 7
        return average

    TMP_DF['VALUE_LOGA'] = TMP_DF.apply(lambda row: calc_ave(row,
                                                             TMP_DF),
                                        axis=1).fillna(0)
    DOMAIN104 = DOMAIN104.append(TMP_DF)

DOMAIN104.to_csv(os.path.join(OUTPUT_FOLDER, '104.csv'), sep=";")
if USE_MONGO:
    bob.mongo.save_df(DOMAIN104, '104', DB_APP, True)
DOMAIN104

In [ ]:
import plotly.graph_objects as go

FIG = go.Figure()

DFD = DOMAIN104[DOMAIN104['KPI'] == 'Confirmed']
FIG = px.line(DFD,
              x="DATE",
              y="VALUE_LOGA",
              color='ENTITY',
              title='Daily trend covid-19')
FIG.update_yaxes(type="log")
FIG.show()